In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [5]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [20]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [34]:
def calc_logloss(y, y_pred):
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
  return err

In [35]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

### Задание 1
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [31]:
def calc_logloss_(y, y_pred):
    err = - np.mean(y * np.log(y_pred, where=(y_pred != 0)) + (1.0 - y) * np.log(1.0 - y_pred, where = ((1.0 - y_pred) != 0)))
    return err

In [32]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss_(y1, y_pred1)

0.10536051565782628

### Задание 3
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [41]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [42]:
W = eval_model(X_st, y, iterations=5000, alpha=1e-4)

500 [ 0.47749927 -0.19692597  0.64660127  1.4945758 ] 1.127820879406358
1000 [ 0.45886878 -0.25439917  0.64531344  1.46695851] 1.0395158244739489
1500 [ 0.44089887 -0.31020268  0.64391187  1.44036779] 0.9565773724088873
2000 [ 0.4236874  -0.36359967  0.6425439   1.41506965] 0.8807599507543653
2500 [ 0.40735199 -0.41357856  0.6414242   1.39140696] 0.8141924355796999
3000 [ 0.39201058 -0.45900098  0.64080604  1.36974771] 0.758759980653503
3500 [ 0.37774317 -0.49894982  0.64090828  1.35037357] 0.7151777920985422
4000 [ 0.36455825 -0.53307004  0.64184068  1.33337513] 0.6825375010766789
4500 [ 0.35239197 -0.56162274  0.6435874   1.31863951] 0.6587997057955899
5000 [ 0.34113653 -0.58526221  0.64604926  1.30592474] 0.6416846622934699


In [43]:
def calc_pred_proba(X, W):
    y_pred_proba = np.floor(sigmoid(np.dot(W, X)))
    return y_pred_proba

In [53]:
calc_pred_proba(X, W)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### Задание 4

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [45]:
def calc_pred(W, X, threshold):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    y_pred[y_pred > threshold] = 1
    y_pred[y_pred < 1] = 0
    return  y_pred

In [46]:
y_pred = calc_pred(W, X_st, 0.5)
y_pred

array([1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])

In [47]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

### Задание 5
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.¶


In [48]:
def accuracy_(y_pred, y):
    return len(y_pred[y == y_pred]) / len(y_pred)

accuracy_(y_pred, y)

0.6

In [51]:
def correlation_matrix(y_pred, y):
    TP = len(y_pred[(y == y_pred) & (y_pred == 1)])
    FN = len(y_pred[(y != y_pred) & (y_pred == 0)])
    FP = len(y_pred[(y != y_pred) & (y_pred == 1)])
    TN = len(y_pred[(y == y_pred) & (y_pred == 0)])
    corr_matrix = pd.DataFrame.from_dict(
        {'y_pred': ['a(x) = +1', 'a(x) = -1'], 
         'y = +1': [TP, FN], 
         'y = -1': [FP, TN]}).set_index('y_pred')
    return corr_matrix

correlation_matrix(y_pred, y)

,y = +1,y = -1
y_pred,,
a(x) = +1,5,4
a(x) = -1,0,1


In [55]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_pred)

array([[1, 4],
       [0, 5]])

In [56]:
def precision_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 1)]))

precision_(y_pred, y)

0.5555555555555556

In [57]:
def recall_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 0)]))

recall_(y_pred, y)

1.0

In [58]:
def F1_(y_pred, y):
    return 2 * precision_(y_pred, y) * recall_(y_pred, y) / (precision_(y_pred, y) + recall_(y_pred, y))

F1_(y_pred, y)

0.7142857142857143